In [ ]:
# %pip install pytest ipytest

## Setup

In [ ]:
import pytest
import ipytest

# Makes the "%%ipytest cell magick work, set options"
# or just call `ipytest.autoconfig()` without options to make it all work easily.
ipytest.config(
    magics=True, 
    defopts="auto", 
    addopts=[
        "-q",  # quiet output
        "-W", "ignore:Module already imported so cannot be rewritten:pytest.PytestAssertRewriteWarning",
    ],
    coverage=False
)

## Topics

### 1. Unit Testing with `pytest`

Anything that your code can do, you can check.  If you find yourself manually checking for something, putting that check in an automated test helps you continue doing it as you develop your project further, automatically.

`pytest` contains a test runner that looks for automated tests; one way it finds them is by looking for function names that start with the word `test_`, in file names that start with the word `test_`.  It runs each function it finds, and marks down whether running it:
  - **Passed**: The function ran with no errors
  - **Failed**: The function ran with an `AssertionError`
  - **Errored**: The function ran with any other error type.

Let's try it out!  Here we'll be writing **"Unit Tests"**, which check that some function or class is working properly (e.g. a function returns the expected thing when called).  Since coding projects tend to be made up of lots of custom functions, it's good to know that the individual custom functions each work the way they should.


**Task**: One of the tests below has an error in it, and so the test is failing. Use the output from pytest to find the failing test and fix it so all tests pass.

In [ ]:
%%ipytest

def test_sum_1_2_is_3():
    assert sum([1, 2]) == 3


def test_sum_2_3_is_5():
    assert sum([1, 2]) == 4



**Task**: Below are three unit tests that check for three different types of things: a value, a type, and an error.  Edit the code so all tests do their intended checks successfully.

In [ ]:
%%ipytest

def test_sum_3_4_is_7():
    # Hint: assert sum([..., ...]) == ...
    raise NotImplementedError()

def test_sum_of_ints_is_an_int():
    # Hint: assert isinstance(sum([...]), int)
    raise NotImplementedError()

def test_sum_strings_a_b_raises_typeerror():
    raise NotImplementedError()
    with pytest.raises(TypeError):
        ... # Put code that should result in an error here.
    
    

## Test Parameterization: Check More Cases with Less Code

Of course, writing a function for every single set of inputs we want to check for is needlessly verbose.  `Parametrizing` tests functions makes that code more condensed, and PyTest provides a decorator for doing this `@pytest.mark.parametrize()`



**Task**: Add two more checks (a.k.a. "test cases") to the tests below, so that a total of 4 tests run:
  - `3 + 7 = 10`
  - `-2 + 3 = 1`

In [ ]:
%%ipytest

cases = [
    [[1, 2], 3],
    [[2, 3], 5],
]
@pytest.mark.parametrize('inputs,output', cases)
def test_sum_of_integers(inputs, output):
    assert sum(inputs) == output


**Task**: Rewrite the three test functions below into a single test function, using `parametrize` to continue checking each case individually. Note that pytest includes an `approx()` function for helping check floats, since there are often little rounding errors with them.

In [ ]:
%%ipytest

def test_5p2_minus_2p1_is_3p1():
    assert 5.2 - 2.1 == pytest.approx(3.1)

def test_6p5_minus_1p7_is_4p8():
    assert 6.5 - 1.7 == pytest.approx(4.8)

def test_0p3_minus_0p2_is_0p1():
    assert 0.3 - 0.2 == pytest.approx(0.1)

## Checking Equality of Numpy Arrays with `numpy.testing` and Pandas DataFrames with `pandas.testing`

**Task**: Write a unit test to check that the computed numpy array is the expected one.  When the test fails, use the error messages to fix the test so that it passes.

In [89]:
%%ipytest

import numpy as np
import numpy.testing as npt
# npt.assert_array_equal()

a = np.array([1, 2, 3])
b = np.array([4, 5, 6])
expected = np.array([5, 7, 8])
observed = a + b
npt.assert_array_equal(expected, observed)


AssertionError: 
Arrays are not equal

Mismatched elements: 1 / 3 (33.3%)
Max absolute difference among violations: 1
Max relative difference among violations: 0.11111111
 ACTUAL: array([5, 7, 8])
 DESIRED: array([5, 7, 9])


no tests ran in 0.02s


**Task**: Write a unit test to check whether the two methods below produce the same dataframes.  When the test fails, use the error messages to fix the test so that it passes.

In [91]:
%%ipytest

import pandas as pd
import pandas.testing as pdt
# pdt.assert_frame_equal()

# Method one: from dictionary
df1 = pd.DataFrame({'a': [1, 2, 3], 'b': [10, 11, 12]})

# Method two: Stepwise DataFrame Mutation
df2 = pd.DataFrame()
df2['b'] = [10, 11, 12]
df2['a'] = [1, 3, 3]

pdt.assert_frame_equal(df1, df2)

AssertionError: DataFrame.columns are different

DataFrame.columns values are different (100.0 %)
[left]:  Index(['a', 'b'], dtype='object')
[right]: Index(['b', 'a'], dtype='object')
At positional index 0, first diff: a != b


no tests ran in 0.01s


## Property Testing with `hypothesis`

There are also cases where you want to check a bunch of inputs to make sure that the code works as correctly, but:
  -  you don't know exactly which inputs are the best to check,
  -  and you aren't sure exactly how to calculate the expected result,
  -  but you know what aspect of the result you want to check (i.e. "property" you want the result to have).

This is called "**Property Testing**", and the `hypothesis` library helps with that.  Just describe the inputs that should go in, and write your test, and it will check your code with a wide range of inputs!

**Task**: The test function below isn't checking what it means to be (as described by the function name), and so Hypothesis keeps finding sets of inputs that make the test fail.  Fix the inputs and the test function body, so the test is correct.

In [ ]:
%%ipytest 

from hypothesis import given
from hypothesis import strategies as st
# For the curious, a full list of "strategy" functions (how hypothesis generates inputs): 
# https://hypothesis.readthedocs.io/en/latest/reference/strategies.html


@given(
    st.lists(st.integers(min_value=-10), min_size=0),
)
def test_sum_of_positive_integers_always_a_positive_integer(inputs):
    assert sum(inputs) > 1


**Task**: Have hypothesis generate `float` values in order to test the function below.

In [ ]:
%%ipytest 

from hypothesis import given
from hypothesis import strategies as st

@given(
    # Put a strategy here for the first float
    # Put a strategy here for the second float
)
def test_sum_of_two_floats_is_always_equivalent_to_using_plus_operator(first, second):
    assert sum([first, second]) == pytest.approx(first + second)
